In [1]:
%display latex

from sage.rings.finite_rings.integer_mod import square_root_mod_prime
from sage.rings.number_field.order_ideal import NumberFieldOrderIdeal
from sage.schemes.elliptic_curves.hom_velusqrt import EllipticCurveHom_velusqrt
from sage.schemes.elliptic_curves.weierstrass_morphism import *

def ideal_de_norme(l,f,D):

    #Trouver un idéal de norme l premier : Cf notes de cours Biasse
    #On se place dans un ordre de discriminant D et de conducteur f
    
    if kronecker(D,l) == 1:
        D = Mod(D,l)
        sq = square_root_mod_prime(D,l) #Lien avec Seysen ? Prendre le min des deux racines vu dans N ? (Optionnel)
        sm = Mod(-sq,l)
        sq = ZZ(sq)
        sm = ZZ(sm)
        sq = min(sm,sq)
        #print(sq)
        return NumberFieldOrderIdeal(O,[l, -sq + f*t]) #Forme donnée dans le cours de Biasse
    elif kronecker(D,l) == 0:
        return NumberFieldOrderIdeal(O,[l, f*t])
    else:
        raise "l est inerte"

#La racine utilisée ici n'est pas le coefficient b de la forme quadratique associée.





def forme_de_seysen(l,D):

    #Trouver une forme quadratique primitive associé à un idéal de norme l premier split : Cf article de Seysen, théorème 3.1
    #On se place dans un ordre de discriminant D 

    #On cherche b la plus petite racine mod 4*l par recherche exhaustive.
    
    if kronecker(D,l) == 1:
        b = 0                                         #Partir de 0 peut poser problème ?
        while Mod(b,4*l)^2 != Mod(D,4*l):
            b = b+1
        ql = BinaryQF([l,b, int((b^2 - D)/(4*l))])
        return ql

def ideal_de_seysen(l,D):
    ql = forme_de_seysen(l,D)
    return NumberFieldOrderIdeal(O,ql)





def base_wk(b,dk):

    #Dans K = Q(t) quadratique de discriminant dk, on se donne b = (x + yt)/2 dans OK
    #On veut écrire b dans la base [1,wk], wk = (dk + t)/2
    
    x = b[0]
    y = b[1]
    wx = x - dk*y
    wy = 2*y
    return (wx,wy)

def base_fq1(b,fm,dk,tracef):
    
    # On représente le froebenius fq par le complexe (tracef + fm*t)/2 dans K = Q(t).
    # Il existe s entier tel que fq = fm*wk + s.
    
    s1 = (-fm*dk + tracef)/2
    (x,y) = base_wk(b,dk)
    fx = x*fm - y*s1
    return (fx,y)

def base_fq2(b,fm,dk,tracef):

    # On représente le froebenius fq par le complexe (tracef - fm*t)/2 dans K = Q(t).
    # Il existe s entier tel que -fq = fm*wk + s.
    # Comme fq et fm*wk ne sont pas égaux à un entier prêt, cela semble être la mauvaise représentation de fq pour la normalisation
    
    s2 = (-fm*dk - tracef)/2
    (x,y) = base_wk(b,dk)
    fx = x*fm - y*s2
    return (fx,-y)





def formes_generatrices (D,N,test_norm): #Trouver la borne : Analyse de Jao

#Algo 2 Jao : Trouver une famille génératrice du groupes de classes, vu comme formes quadratiques réduites,
# En n'utilisant que des idéaux de normes premiers petits splits et autorisés pour le calcul d'isogénies horizontales
# On génére simplement une famille de N idéaux, N supposé suffisament grand pour que leurs classes soient génératrices.
# Seysen partie 3 : N = 2log^2(D) suffirait (Admet Riemann Généralisé)
    
    Idéaux = []
    Formes = []
    Primes = []
    i = 2
    while i < N:
        if kronecker(D,i) == 1 and (test_norm)%i != 0:
            Primes.append(i)
            qi = forme_de_seysen(i,D)
            Li = NumberFieldOrderIdeal(O,qi)
            Idéaux.append(Li)
            Formes.append(qi)
        i = next_prime(i)
    return Idéaux, Formes, Primes



def facto_complet(facto,Primes):
    
    #On veut savoir si un entier a de factorisation facto = factor(a) ne contient que des premiers de la liste Primes
    # On utilise le fait que facto et Primes soit triée

    k = len(facto)
    if len(Primes) >= k:
        i = 0
        j = 0
        while i < k and j < len(Primes):
            if facto[i][0] == Primes[j]:
                i = i+1
                j = j+1
            else:
                j = j+1
        return i == k
    else:
        return False
            
        

def factorisation(L,D,N,Borne_z,test_norm):

    #Algo 3 Jao : (Hafney / McCurley) Factoriser L dans la base précédente
    #On cherche au hasard à ramener la classe de L à une classe simplifiable par le théorème 3.1 de Seysen.
    #On représente les classes par des formes quadratiques réduites : c'est nécessaire pour se ramener à Seysen
    #Borne_z est une borne donnée par Jao dans l'analyse de l'algorithme. Le nombres de facteurs de L ne dépassera pas Borne_z
    
    Idéaux, Formes, Primes = formes_generatrices (D,N,test_norm)
    print('liste des nombres premiers autorisées')
    print(Primes)
    ql = L.quadratic_form().reduced_form()
    print(ql)                               #représente la classe de L
    k = len(Primes)
    
    a = ql[0]
    facto = factor(a)
    
    premier_utilisé = [] # On garde en mémoire la liste des nombres premiers intervenant dans la factorisation de L
    relation = ql        # Partant de ql, on va chercher une classe simplifiable
    
    expx = [0]*k
    liste_indice = []
    while facto_complet(facto,Primes) == False:
        
        # Choix d'exposant au hasard, on veut entre 3 et Borne_z coefficient non nul (Cf algo 3 Jao)
        # Autoriser 1 ou 2 comme nombre d'exposant possible ?
        
        print('tentative indices')
        expx = [0]*k       # La liste des exposants notée xi par Jao
        nb_indice = randint(3,Borne_z) 
        liste_indice = []    # on choisit les idéaux qui vont intervenir dans la factorisation, au hasard.
        while len(liste_indice) < nb_indice:
            i = randint(0,k-1)
            if i not in liste_indice :
                liste_indice.append(i)
        print(nb_indice)
        print(liste_indice)
        
        for i in [0 .. k-1]:
            if i in liste_indice:
                expi = randint(1,int((N/Primes[1])^2))   #On attribue à chaque indice un exposant xi, bornée d'après Jao.
                expx[i] = expi

        print('exposants associées')
        print(expx)
        
        # calcul de l*( fi^exp(xi) pour tout i ) pour chercher une relation "simplifiable"
        relation = ql
        for i in [0 .. k-1]:
            if expx[i] != 0 :
                facteur = Formes[i]
                if expx[i]>1 :
                    for _ in range (expx[i]-1) :
                        facteur = Formes[i]
                        facteur = facteur.reduced_form()
                relation = ql*facteur
                relation = relation.reduced_form()
        a = relation[0]
        facto = factor(a) #Si a factorise complétement dans Primes, c'est gagné avec Seysen

    print('relation simplifiable trouvée')
    print(relation)

    #On retient les nombres premiers utilisés :
    for i in liste_indice:
        premier_utilisé.append(Primes[i])

    print('liste des nombres premiers utilisés jusque là')
    print(premier_utilisé)

    print('Factorisation de a')
    print(facto)
    
    
    #Calcul des valuations de a en chaque Primes, et change le signe selon la méthode Seysen
    expu = []     #liste des exposant ui de Jao
    ka = len(facto)
    b = relation[1]
    i = 0
    for j in [0 .. k-1]:
        if i < ka :
            if Primes[j] == facto[i][0] :
                if facto[i][0] not in premier_utilisé :
                    premier_utilisé.append(facto[i][0])
                expu.append(facto[i][1])
                i = i + 1
                bi = Formes[i][1]
                if Mod( b , 2*Primes[i]) != Mod( bi , 2*Primes[i]):
                    expu[-1] = -expu[-1]
            else :
                expu.append(0)
        else :
            expu.append(0)
                        

    #inverser la relation pour trouver L:
    expe = []     #liste des exposants ei de Jao
    factoL = []
    for j in [0 .. k-1] :
        ej = expu[j] - expx[j]
        if ej > 0 :
            expe.append(ej)
            factoL.append(Idéaux[j].conjugate())
        elif ej < 0 :
            expe.append(-ej)
            factoL.append(Idéaux[j])

    return expe, factoL, premier_utilisé

def eval_fq(P,q,E):

    #calcul Frobenius(P) sur une courbe E définie sur Fq
    #Le point P peut appartenir à un extension, il faut alors donner E telle que P appartienne à E.
    
    Q = [P[0],P[1]]
    Q[0] = (Q[0])^101
    Q[1] = (Q[1])^101
    Q = E(Q)
    return Q


#Construction du groupe de torsion E[L]
#Pour trouver E[L], besoin de c et d : deux choix selon fq1 ou fq2 à priori, on va tout de même privilégier le choix 1.

def noyaux(c,d,P,Q,q,E):
    
    #Q est dans E[L] ssi dans E[l] et dans E[c+dfd]. On teste pour chaque P +iQ générateur des sous groupes de E[l] d'ordre l. 
    
    for i in [0 .. l-1]:
        Pi = P + i*Q
        cPi = c*Pi
        fPi = eval_fq(Pi,q,E)
        dfPi = d*fPi
        pPi = cPi + dfPi
        if pPi == E(0,1,0):
            return Pi
    cQ = c*Q
    fQ = eval_fq(Q,q,E)
    dfQ = d*fQ
    pQ = cQ + dfQ
    if pQ == E(0,1,0):
        return Q
    else:
        return None

def groupes_torsions(E,L,l,q,fm,dk,tracef):

    #On veut trouver un générateur de E[L].
    #On mets en arguments des données du problèmes précalculées.
    
    #On écrit L dans la base [1,fq]
    cible = (L.gens_two())[1]
    (c1,d1) = base_fq1(cible, fm, dk, tracef)
    (c2,d2) = base_fq2(cible, fm, dk, tracef) #Inutile sauf si fq2 peut être la bonne représentation du frobenius.

    #On trouve des générateurs de E[l]
    Fl = E.division_field(l)          #ETAPE TROP LENTE, le degrée de l'extension est de l'odre de l d'après l'article de Broker.
    El = EllipticCurve(Fl, [E.a4(),E.a6()])
    G1, G2 = El.torsion_basis(l)

    #On en déduit des générateurs de E[L] selon le choix de fq1 ou fq2. Dans la suite on ne conserve que fq1.
    return El, noyaux(c1,d1,G1,G2,q,El), noyaux(c2,d2,G1,G2,q,El)





def phi_from_ker(K,l,E):

    #On calcul une isogénie de degré l et noyau K, normalisée. 
    
    if l >= 9:
        phi = EllipticCurveHom_velusqrt(E, K) #Square vélu n'est pas implémenté pour l < 10. Vor plus lent pour l < 100 ?
    else:
        phi = EllipticCurveIsogeny(E, K) #Applique la formule de vélu. Ne renvoie pas le même type que square vélu, mais compatible.

    #normalisation :
    u = phi.scaling_factor() #calcul l'action sur l'invariant différentiel
    E1 = phi.codomain()
    isom = WeierstrassIsomorphism(E1, (u^(-1), 0, 0, 0)) #Corrige la courbe d'arrivée 
    phi_normal = isom * phi
    return phi_normal



def phi_from_L(E,L,l,q, fm, dk, tracef):

    #résume ce qui précéde pour passer d'un idéal de norme l à une isogénie de degré l.
    
    El, K1, K2 = groupes_torsions(E,L,l,q,fm, dk, tracef)
    phi = phi_from_ker(K1,l,El)
    return phi






def composantes_de_phi(E,q,Exposant,Idéaux,fm,dk,tracef):

    #Algo 4 de Jao (Algo de Broker) : On déduit de la factorisation de L une suite d'isogénies.
    
    dépard = E   #Le point de dépard sera en réalité E_eval car on souhaite évaluer un point Q. 
    k = len(Idéaux)
    composantes = []
    for i in [0 .. k-1]:
        pi = Idéaux[i]
        print('Composante de degré :')
        print(pi.norm())
        print('Exposant :')
        print(Exposant[i])
        for _ in range(Exposant[i]):
            phic = phi_from_L(dépard,pi,pi.norm(),q,fm,dk,tracef)
            composantes.append(phic)
            dépard = phic.codomain()
            dépard = EllipticCurve(Fq, [dépard.a4(),dépard.a6()]) #On se replace sur le corps de base pour éviter une escalade d'extensions
    return composantes



#Gestion de l'idéal principal :

def Cornacchia_avec_racine(n,d,s):

    #On veut trouver un élément (x + y*f*t)/2 dans l'idéal, de norme n, à partir d'une racine s du discrimant D, s déjà obtenue.
    #Jao et Hardy : Cornacchia général : on résoud 4*n = x²+y²|D| ( Avec les notations de Hardy, f = 1, g = |D|, m = D )
    #Il faut à priori que 4*n et D soit premier entre eux ... ne semble pas garanti. 

    #On applique l'étape 6 de l'algorithme d'Hardy, c'est à dire l'étape de division euclidienne de Cornacchia
    
    while s > int(sqrt(4*n)) :
        r = n%s
        n = s
        s = r
    x = s
    y = (4*n - s^2)/abs(d)
    if y.is_square():
        y = int(sqrt(y))
        return (x,y)
    else :
        return (0,0)

def calcul_de_alpha(L,Idéaux,Exposant,D):
    B = L     # B sera l'idéal principal dont on cherche un générateur.
    k = len(Idéaux)
    m = 1    #ici m désigne le produit des normes des idéaux premier qui factorise L. alpha = beta/m
    for i in [0 .. k-1] :
        pi = Idéaux[i]
        for _ in range (Exposant[i]):
            B = B*(pi.conjugate())
            m = m*(pi.norm())

    if B.is_principal():
        n = B.norm()
        qB = B.quadratic_form()
        s = qB[1]                   # s est une racine de D modulo 4*n car qB est de discriminant D
        x, y = Cornacchia_avec_racine(n,D,s)
        beta = (x + y*t)/2 
        bO = NumberFieldOrderIdeal(O,beta)
        if bO == B:
            return beta, m
        else:
            beta = (x - y*t)/2
            return beta, m
    else:
        raise 'Erreur de factorisation'



def eval_phic(Q,Composantes):
    #On calcul l'image d'un point Q par la suite de composantes de phi.
    #Q appartient à E_eval, donc potentiellement pas à E(Fq). 
    for phi in Composantes:
        Q = phi(Q)
    return Q

def isom_normalisation(Ec,ua,va,m,fm):
    #On calcul l'isomorphisme qui corrige l'a courbe d'arrivée, pour normaliser phi. 
    #Il suffit de corriger l'action de [alpha] car les composantes sont normalisées.
    # alpha = (ua + va fq )/(fm*m)
    u = Fp(ua)*((Fp(m*fm))^-1)
    isom = WeierstrassIsomorphism(Ec, (u^(-1), 0, 0, 0))
    return isom

def endo_alpha(Q,En,q,u,v,m,fm,Cn):
    #On veut calculer l'image d'un point Q par l'endomorphisme alpha. Q appartient à la courbe d'arrivée des composantes, normalisée. 
    #alpha = (u + v fq)/(mfm)
    denom = (Mod(m,Cn)*Mod(fm,Cn))^(-1) #inversible par choix des nombres premiers depuis le début. 
    c = Mod(u,Cn)*denom
    d = Mod(v,Cn)*denom
    cQ = c*Q      #a bien un sens car c est un entier
    fqQ = eval_fq(Q,q,En)
    dfqQ = d*fqQ
    alphaQ = En(cQ) + En(dfqQ) # ici correction d'un erreur de type inconnue, en précisant que l'on veut additionner des points de En
    return alphaQ


In [2]:
def Broker(E,p,q,E_eval,Q,n,K,O,l,N,Borne_z):

    #Calcul des données du problème :
    dk = K.discriminant()
    f = O.conductor()
    D = f^2*dk
    
    CE = E.cardinality_pari()
    tracef = E.trace_of_frobenius()
    Dm = tracef^2 - 4*q
    fm = sqrt(Dm/(K.discriminant()))
    
    Cn = E_eval.cardinality()
    test_norm = fm*p*Cn   #On souhaite éviter tout nombre premier qui divise test_norm

    #Application des algos précédents :
    
    L = ideal_de_norme(l,f,D)
    
    print('début factorisation')
    Exposant, Idéaux, premier_utilisé = factorisation(L,D,N,Borne_z,test_norm)
    print('fin factorisation')
    print(premier_utilisé)
    print(Exposant)
    
    print('début Composantes')
    Composantes = composantes_de_phi(E_eval,q,Exposant,Idéaux,fm, dk ,tracef)
    print('fin Composantes')
    
    beta, m = calcul_de_alpha(L,Idéaux,Exposant,D)
    u, v = base_fq1(beta , fm , dk, tracef)

    Ec = Composantes[-1].codomain()
    Qc = eval_phic(Q,Composantes)
    
    isom = isom_normalisation(Ec,u,v,m,fm)
    Qc = isom(Qc)
    E_phi = isom.codomain()

    Qc = endo_alpha(Qc, E_phi,q,u,v,m,fm,Cn)

    return Qc, E_phi

def sqrtvelu(E,q,E_eval,K,O,l):
    
    dk = K.discriminant()
    f = O.conductor()
    D = f^2*dk
    tracef = E.trace_of_frobenius()
    Dm = tracef^2 - 4*q
    fm = Dm/(K.discriminant())
    
    L = ideal_de_norme(l,f,D)
    El, K1, K2 = groupes_torsions(E_eval,L,l,q,fm, dk, tracef)
    phi = phi_from_ker(K1,l,El)
    
    return phi

In [3]:
#Exemple de Broker "small"

p = 101 
q = p^1
Fp = GF(p)
Fq = GF(q)
E = EllipticCurve(Fq, [79,44])

K.<t> = QuadraticField(-179)
O = K.maximal_order() #Calculer O et K à partir de E ?

N = 30
z = 1/(2*sqrt(3))                        #z théorique de Jao
Borne_z = int(sqrt(log(179/3,2))/z)     #Proposition de borne de Jao

n = 1
Fqn.<xn> = GF(q^n)
E_eval = EllipticCurve(Fqn, [79,44])
P = E_eval.random_point()

#coordonée d'un élément du noyau dans F101^15 :
#88*t^14 + 80*t^13 + 27*t^12 + 33*t^11 + 29*t^10 + 96*t^9 + 9*t^8 + 47*t^7 + 73*t^6 + 42*t^5 + 67*t^4 + 66*t^3 + 89*t^2 + 27*t + 26
#100*t^14 + 44*t^13 + 33*t^12 + 98*t^11 + 91*t^10 + 62*t^9 + 99*t^8 + 14*t^7 + 65*t^6 + 56*t^5 + 69*t^4 + 61*t^3 + 39*t^2 + 88*t + 23

l = 31

Q , E_phi = Broker(E,p,q,E_eval,P,n,K,O,l,N,Borne_z)
Q

début factorisation
liste des nombres premiers autorisées
[5, 13, 17, 19]
5*x^2 + x*y + 9*y^2
relation simplifiable trouvée
5*x^2 + x*y + 9*y^2
liste des nombres premiers utilisés jusque là
[]
Factorisation de a
5
fin factorisation
[5]
[1]
début Composantes
Composante de degré :
5
Exposant :
1
fin Composantes


(40 : 89 : 1)

In [4]:
E_phi

Elliptic Curve defined by y^2 = x^3 + 96*x + 75 over Finite Field in t of size 101^4

In [5]:
phi = sqrtvelu(E,q,E_eval,K,O,l)
phi.codomain()

Elliptic Curve defined by y^2 = x^3 + 96*x + 75 over Finite Field in t of size 101^15

In [6]:
phi(P)

(40 : 89 : 1)

In [8]:
phi(P) == Q # Les deux résultats ne sont pas vu dans la même extension

False

In [10]:
#Exemple de Broker "medium"

p = 99999999980010207001
q = p^1
Fp = GF(p)
Fq = GF(q)
E = EllipticCurve(Fq, [93111780581619358815, 13776438796781696372])

K.<t> = QuadraticField(-3635)
O = K.maximal_order()     #Calculer O et K à partir de E ?

#Bornes pour la factorisation :
z = 1/(2*sqrt(3))                        #z théorique de Jao
Borne_z = int(sqrt(log(3635/3,2))/z)     #Proposition de borne de Jao
N = 2*int(log(3635,2))^2                 #Borne théorique de Seysen

n = 1
Fqn.<xn> = GF(q^n)
E_eval = EllipticCurve(Fqn, [93111780581619358815, 13776438796781696372])
P = E_eval(73931099962253475826, 29177286940991158970)

l = next_prime(10^21)

#Q , E_phi = Broker(E,p,q,E_eval,P,n,K,O,l,N,Borne_z)   
#L'application de l'algo trouve une factorisation en quelques secondes, mais bloque sur le calcul des composantes